# Coleta de dados para treinamento de modelo de espectro político para NLU

Modelo de classificação de espectro politico, a partir de noticias

### Direita

- Jovem Pan

### Esquerda

- Carta Capital

In [ ]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re

training_data = []

## Jovem Pan

In [ ]:
links = []
for page in tqdm(range(0, 15)):
    html = urlopen(f"https://jovempan.com.br/noticias/politica/page/{page}")
    bs = BeautifulSoup(html, 'html.parser')
    linhas = bs.find_all('h2', {'class':'post-title'})
    for i in linhas:
        links.append(str(i.contents[0]).split('"')[1])
print(len(links))

In [ ]:
for link in tqdm(links):
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    linhas = bs.find_all('div', {'class':'context'})
    text = ""
    for i in linhas:
        for t in i.findChildren("p"):
            p = re.compile(r'<.*?>')
            res = p.sub('', str(t))
            text += res
    n = 2000 # chunk length
    chunks = [{"text": text[i:i+n], "labels": ["Direita"]} for i in range(0, len(text), n)]
    training_data.extend(chunks)
print(len(training_data))

## Carta Capital

In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome(executable_path=r'./chromedriver.exe')
driver.get('https://www.cartacapital.com.br/mais-recentes/')
time.sleep(3)

### Pegando os links

In [ ]:
# Não precisa dessa parte usando o Jupyter

def scroll_down():
    scroll_pause_time = 2 
    i = 3

    while True:
        screen_height = driver.execute_script("return window.screen.height;")
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
        i += 3
        time.sleep(scroll_pause_time)

        scroll_height = driver.execute_script("return document.body.scrollHeight;")

        # Após descer a página ele clica no botão ler mais
        try: 
            btn_load_more = driver.find_element_by_class_name("eltdf-bnl-load-more")
            btn_load_more.click()

            time.sleep(scroll_pause_time * 3)
        except:
            pass

        if (screen_height) * i > scroll_height:
            break

j = 0
while j < 5:
    scroll_down()
    j += 1

In [ ]:
bs = BeautifulSoup(driver.page_source, 'html.parser')
mat = bs.find("div", class_="eltdf-bnl-outer")
links = [h3.find("a").get("href") for h3 in mat.find_all("h3")]

with open('links.txt', 'w') as file: 
    for link in links:
        content = link + '\n' 
        file.write(content)

### Pegando Conteudo

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

In [ ]:
with open('links.txt', 'r') as file:
    links = [line[:-1] for line in file.readlines()]
    file.close()

In [ ]:
lista = ['Um minuto, por favor...', 'Obrigado por ter chegado até aqui. Combater a desinformação, as mentiras e os ataques às instituições custa tempo e dinheiro. Nós, da CartaCapital, temos o compromisso diário de levar até os leitores um jornalismo crítico, alicerçado em dados e fontes confiáveis. Acreditamos que este seja o melhor antídoto contra as fake news e o extremismo que ameaçam a liberdade e a democracia.', 
'Se você acredita no nosso trabalho, junte-se a nós. Apoie, da maneira que puder. Ou assine e tenha acesso ao conteúdo integral de CartaCapital!', 'ou, se preferir,']

for url in links:
    r = requests.get(url).content
    soup = BeautifulSoup(r, 'html.parser')

    article = soup.find("article")
    ps = article.find_all("p")

    strings = [p.string for p in ps if p.string != None and p.string not in lista]
    text = "".join(strings)

    n = 2000 # chunk length
    chunks = [{"text": text[i:i+n], "labels": ["Esquerda"]} for i in range(0, len(text), n)]
    training_data.extend(chunks)

## Salva Arquivo

In [ ]:
training_data_filename = 'training_data.json'
with open(training_data_filename, 'w+', encoding='utf-8') as f:
    json.dump(training_data, f, indent=4, ensure_ascii=False)

print('Data successfully saved locally in ' + training_data_filename)